In [1]:
THRESHOLD_INDICATING_SHUTDOWN = 5
MISSING_VALUE = -999
OPERATION_VAL_RANGE = (713.682, 763.826)

In [2]:
def remove_long_shutdown(numbers, num_consecutive, missing_label):
    chunks = []
    current_chunk = []

    i = 0
    while i < len(numbers)-1:
        num = numbers[i]
        if num != missing_label:
            current_chunk.append(i)
        else:
            j = i+1
            while j < len(numbers):
                if numbers[j] == missing_label:
                    j += 1
                else:
                    break

            if j-i < num_consecutive:
                current_chunk += range(i,min(j+1, len(numbers)))# numbers[i:j+1]
            else:
                chunks.append(current_chunk)
                current_chunk = []

            i=j         

        i+= 1

    # Append the last chunk
    if current_chunk:
        chunks.append(current_chunk)

    to_ret = []
    # Print the chunks
    for i, chunk in enumerate(chunks, 1):
        to_ret += chunk
        
    return to_ret

In [3]:
import os
import pandas as pd
import numpy as np
import sklearn
from sklearn import metrics
import traceback

def calculate_f1(path_to_result, label_file):
    for result_file in os.listdir(path_to_result):
        try:
            print('----------', result_file, '----------')
            result_adapad = pd.read_csv(path_to_result + '/' + result_file)
            result_adapad = result_adapad.dropna()
            result_adapad = result_adapad.reset_index(drop=True)
            print(len(result_adapad))

            label = pd.read_csv(label_file)
            label = label[-len(result_adapad):]
            label = label.reset_index(drop=True)
            print(len(label))
            
            total = pd.concat([result_adapad, label], axis=1, join='inner')
            observed_values = total.observed.values.tolist()
            observed_values = [int(x) for x in observed_values]
            to_keep_comparision = remove_long_shutdown(observed_values, THRESHOLD_INDICATING_SHUTDOWN, MISSING_VALUE)
            total=total[total.index.isin(to_keep_comparision)]

            preds = result_adapad.anomalous
            preds = preds.dropna()
            preds = preds.astype(int)

            Precision, Recall, F, Support =metrics.precision_recall_fscore_support(total.is_anomaly.values.tolist(), 
                                                                                   total.anomalous.values.tolist(), 
                                                                                   zero_division=0)
            precision = Precision[1]
            recall = Recall[1]
            f = F[1]
            print(precision, recall, f)
        except:
            pass
        
path_to_result = "/mnt/c/Arbeid/00_Projects/AutoQC_git/AutoQC_framework/AdapAD/Tide_Pressure"
label_file = "/mnt/c/Arbeid/00_Projects/AutoQC_git/anomaly_detection_algs/1-data/Tide_Pressure.org.csv"
calculate_f1(path_to_result, label_file)

---------- bk ----------
---------- progress_0.0002.csv ----------
16692
16692
0.020165289256198347 0.8970588235294118 0.03944390559327514
---------- progress_0.0008.csv ----------
16692
16692
0.14473684210526316 0.8088235294117647 0.2455357142857143
---------- progress_0.001.csv ----------
16692
16692
0.2391304347826087 0.8088235294117647 0.3691275167785235
---------- progress_0.0014.csv ----------
16692
16692
0.48672566371681414 0.8088235294117647 0.6077348066298343
---------- progress_0.0018.csv ----------
16692
16692
0.6585365853658537 0.7941176470588235 0.72
---------- progress_0.0019.csv ----------
16692
16692
0.7066666666666667 0.7794117647058824 0.7412587412587412
---------- progress_0.002.csv ----------
16692
16692
0.7123287671232876 0.7647058823529411 0.7375886524822695
---------- progress_0.0022.csv ----------
16692
16692
0.7083333333333334 0.75 0.7285714285714285
---------- progress_0.0026.csv ----------
16692
16692
0.7419354838709677 0.6764705882352942 0.7076923076923077
-

In [4]:
import os
import pandas as pd
import numpy as np
import sklearn
from sklearn import metrics
import traceback
import math

def sigmoid(x):
    return 1 / (1 + math.exp(-x))

def calculate_roc(path_to_result, label_file):
    for result_file in os.listdir(path_to_result):
        try:
            print('----------', result_file, '----------')
            result_adapad = pd.read_csv(path_to_result + '/' + result_file)
            result_adapad = result_adapad.dropna()
            result_adapad = result_adapad.reset_index(drop=True)
            print(len(result_adapad))

            label = pd.read_csv(label_file)
            label = label[-len(result_adapad):]
            label = label.reset_index(drop=True)
            print(len(label))

            # remove shutdown period data
            total = pd.concat([result_adapad, label], axis=1, join='inner')
            observed_values = total.observed.values.tolist()
            observed_values = [int(x) for x in observed_values]
            to_keep_comparision = remove_long_shutdown(observed_values, THRESHOLD_INDICATING_SHUTDOWN, MISSING_VALUE)
            total=total[total.index.isin(to_keep_comparision)]

            # calculate anomaly scores
            total["anomaly_score"] = np.ones((len(total), 1))
            normal_observed_range = total[(total.observed >= OPERATION_VAL_RANGE[0]) & 
                                          (total.observed <= OPERATION_VAL_RANGE[1])]
            #normal_observed_range
            anomaly_scores_normal_condition = normal_observed_range.err - normal_observed_range.threshold
            anomaly_scores_normal_condition = anomaly_scores_normal_condition.values.tolist()
            anomaly_scores_normal_condition = [sigmoid(x) for x in anomaly_scores_normal_condition]
            total.loc[normal_observed_range.index, 'anomaly_score'] = anomaly_scores_normal_condition

            roc_auc = metrics.roc_auc_score(total.is_anomaly, total.anomaly_score)
            print("Estimated ROC AUC:", roc_auc)

            y, x, _ = metrics.precision_recall_curve(total.is_anomaly, total.anomaly_score)
            pr_auc = metrics.auc(x, y)
            print("Estimated PR AUC:", pr_auc)
        except:
            pass
        
path_to_result = "/mnt/c/Arbeid/00_Projects/AutoQC_git/AutoQC_framework/AdapAD/Tide_Pressure"
label_file = "/mnt/c/Arbeid/00_Projects/AutoQC_git/anomaly_detection_algs/1-data/Tide_Pressure.org.csv"
calculate_roc(path_to_result, label_file)

---------- bk ----------
---------- progress_0.0002.csv ----------
16692
16692
Estimated ROC AUC: 0.9007944895876946
Estimated PR AUC: 0.744927832590005
---------- progress_0.0008.csv ----------
16692
16692
Estimated ROC AUC: 0.9347843760543398
Estimated PR AUC: 0.7529206812811994
---------- progress_0.001.csv ----------
16692
16692
Estimated ROC AUC: 0.9340984199636597
Estimated PR AUC: 0.7522443771173056
---------- progress_0.0014.csv ----------
16692
16692
Estimated ROC AUC: 0.9337254630965787
Estimated PR AUC: 0.7496338432217032
---------- progress_0.0018.csv ----------
16692
16692
Estimated ROC AUC: 0.9336604242968696
Estimated PR AUC: 0.7497228872840295
---------- progress_0.0019.csv ----------
16692
16692
Estimated ROC AUC: 0.9336533106781515
Estimated PR AUC: 0.7497286193412203
---------- progress_0.002.csv ----------
16692
16692
Estimated ROC AUC: 0.9336461970594333
Estimated PR AUC: 0.75122881502395
---------- progress_0.0022.csv ----------
16692
16692
Estimated ROC AUC: 0.93

In [5]:
path_to_result = "/mnt/c/Arbeid/00_Projects/AutoQC_git/AutoQC_framework/AdapAD/Wave_Height"
label_file = "/mnt/c/Arbeid/00_Projects/AutoQC_git/anomaly_detection_algs/1-data/Wave_Height.label.final.csv"
OPERATION_VAL_RANGE = (0, 15.2)
calculate_f1(path_to_result, label_file)

---------- bk ----------
---------- progress_0.05.csv ----------
27962
27962
0.9572713643178411 0.995323460639127 0.9759266335498663
---------- progress_0.1.csv ----------
27962
27962
0.978494623655914 0.9929851909586905 0.9856866537717601
---------- progress_0.12.csv ----------
27962
27962
0.9822393822393822 0.9914263445050663 0.9868114817688131
---------- progress_0.13.csv ----------
27962
27962
0.9837587006960556 0.9914263445050663 0.9875776397515528
---------- progress_0.14.csv ----------
27962
27962
0.9852827265685515 0.9914263445050663 0.9883449883449884
---------- progress_0.15.csv ----------
27962
27962
0.9875679875679876 0.990646921278254 0.9891050583657587
---------- progress_0.16.csv ----------
27962
27962
0.9914129586260734 0.9898674980514419 0.9906396255850234
---------- progress_0.17.csv ----------
27962
27962
0.9929632525410477 0.9898674980514419 0.9914129586260734
---------- progress_0.18.csv ----------
27962
27962
0.9929632525410477 0.9898674980514419 0.991412958626073

In [6]:
path_to_result = "/mnt/c/Arbeid/00_Projects/AutoQC_git/AutoQC_framework/AdapAD/Wave_Height"
label_file = "/mnt/c/Arbeid/00_Projects/AutoQC_git/anomaly_detection_algs/1-data/Wave_Height.label.final.csv"
OPERATION_VAL_RANGE = (0, 15.2)
calculate_roc(path_to_result, label_file)

---------- bk ----------
---------- progress_0.05.csv ----------
27962
27962
Estimated ROC AUC: 0.9999133183120673
Estimated PR AUC: 0.999453054162743
---------- progress_0.1.csv ----------
27962
27962
Estimated ROC AUC: 0.9999043297592117
Estimated PR AUC: 0.9994019160206933
---------- progress_0.12.csv ----------
27962
27962
Estimated ROC AUC: 0.9998948072329193
Estimated PR AUC: 0.9993471518527619
---------- progress_0.13.csv ----------
27962
27962
Estimated ROC AUC: 0.9998979220779682
Estimated PR AUC: 0.9993638390488029
---------- progress_0.14.csv ----------
27962
27962
Estimated ROC AUC: 0.999129267315468
Estimated PR AUC: 0.9987466357327032
---------- progress_0.15.csv ----------
27962
27962
Estimated ROC AUC: 0.9991290893243225
Estimated PR AUC: 0.9987453774416821
---------- progress_0.16.csv ----------
27962
27962
Estimated ROC AUC: 0.9991278433863029
Estimated PR AUC: 0.9987385111649955
---------- progress_0.17.csv ----------
27962
27962
Estimated ROC AUC: 0.9998309084116301